<a href="https://colab.research.google.com/github/thyungkim/NetDissect-Improve-Accuracy/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageNet
from torchvision import models
from torchvision.datasets import CIFAR100
from torchvision.datasets import Places365



In [4]:
# Load the Places365 dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

root = os.path.expanduser("~/.cache")
train_data = CIFAR100(root, download=True, train=True, transform = transform)
test_data = CIFAR100(root, download=True, train=False, transform = transform)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting /root/.cache/cifar-100-python.tar.gz to /root/.cache
Files already downloaded and verified


In [20]:
train_data = Places365(root, download=True, split='train-standard', transform = transform)
test_data = Places365(root, download=True, split='val', transform = transform)

  0%|          | 0/67498496 [00:00<?, ?it/s]

Extracting /root/.cache/filelist_places365-standard.tar to /root/.cache


HTTPError: ignored

In [5]:
#model = models.vgg16(pretrained = False)
model = models.resnet18(pretrained = False)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
def acc(dataloader, model):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in dataloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  accuracy = correct / total
  return accuracy

In [16]:
def train(model, epoch, optimizer, criterion, dataloader, device):
  for i in range(epoch):
      running_loss = 0.0
      for idx, (inputs, target) in enumerate(dataloader):
          inputs = inputs.to(device)
          target = target.to(device)
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, target)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
      print(f'Epoch {i + 1} loss: {running_loss / (i + 1)}')
  return model

In [17]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import models

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



# Create the VGG-16 model
model = models.vgg16(pretrained=False)
model.to(device)


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model = train(model, 100, optimizer, criterion, trainloader, device)
train_accuracy = acc(trainloader, model)
test_accuracy = acc(testloader, model)


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1 loss: 7219.600012779236
Epoch 2 loss: 3019.649365067482
Epoch 3 loss: 1837.3872783184052
Epoch 4 loss: 1269.696786671877
Epoch 5 loss: 936.9905580997467
Epoch 6 loss: 722.4510177572569
Epoch 7 loss: 570.5090434551239
Epoch 8 loss: 456.1484190374613
Epoch 9 loss: 367.0551915433672
Epoch 10 loss: 293.1692705094814
Epoch 11 loss: 230.1866756081581
Epoch 12 loss: 177.0860649123788
Epoch 13 loss: 133.56971124960825
Epoch 14 loss: 98.52184786328247
Epoch 15 loss: 72.60337212383747
Epoch 16 loss: 54.12070500990376
Epoch 17 loss: 40.53486745037577


KeyboardInterrupt: ignored

In [14]:
torch.cuda.is_available()

True

In [ ]:
trainloader